In [11]:
# imports
from sqlalchemy import create_engine
import pandas as pd

In [12]:
DB_USER = "s1144085"
DB_PASS = "9515"
DB_HOST = "95.217.3.61"
DB_PORT = "5432"
DB_NAME = "minor_s1144085"
SCHEMA_NAME = "INMINDAA_project"

In [13]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [14]:
def select_all_from_dim_hittestress():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_hittestress"}
    """
    return pd.read_sql(query, con=engine)

In [15]:
df = select_all_from_dim_hittestress()
df.head(5)

,hittestress_id,pc4_code,_count,_sum,_mean,pet_gem,rivm_class,jaartal
0,1,9711,42601,1887215,44.299780,44,6,2022
1,2,9712,42601,1887215,44.299780,44,6,2022
2,3,9711,72231,3185994,44.108402,44,6,2022
3,4,9712,72231,3185994,44.108402,44,6,2022
4,5,9711,37722,1713632,45.427920,45,6,2022
